# Example of using `cells_in_gel.batch` for Green (Phalloidin 488) Channel 

The Wrapping Function section shows the quickest way to achieve results using 20x tiff stacks with filenames labeled as:

* Example filename: I61QTTA-CFbs_I61Q5ECM_1mMRGD_20x_002
* Fibroblast Genotype (I61QTTA)
* ECM Genotype and Concentration (I61QTTA, 5mg/mL)
* RGD Concentration (1mM)
* Gel number (1 z-stack acquired per 5uL gel, 3 gels per condition)


The Maximum Intensity Projection, Segmentation and Region Properties, Extracting Average Values from Max Intesnity Projections, and  Graphing Groups sections all describe user tunable functions for more in depth approach of extracting data. 

## Imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob

from skimage.morphology import disk

from cells_in_gel import batch

## Load Images

Load your images from a filepath using `glob.glob('filenames.tif')` into a list of strings. The `*` can be used to capture all files with the remainder of specifications. For example, `*.tif` will capture all tif files in the directory. 

In [ ]:
# list of filenames
files = glob.glob('C3*.tif') 

## Wrapping Function

The wrapping function, `batch.max_proj_wrap(files)`, takes the input filenames, and returns the segmented images with graphs of each comparison group. The grouping is based on comparing with or without RGD present, ECM type, and cell type. 

In [ ]:
batch.max_proj_wrap(files)

Further tunability can be achieved by following the remainder of this notebook as a step-by-step guide to using each function in batch. 

## Maximum Intensity Projections

Maximum intensity projections can be obtained from each z stack and placed into a dictionary using the `batch.max_projection(files)` function. 

In [ ]:
max_proj = batch.max_projection(files)

Here is a plot showing the max intensity projection from the 19th filename in the list of `files`. By changing the number in `files[18]`, you can look through various max projections from the images. 

In [ ]:
plt.imshow(max_proj[files[18]])
plt.axis('off')

## Segmentation and Region Properties

Create the `selem` (shape) for separation of objects. A higher number will result in less separation between objects in max intensity projections.

In [ ]:
selem = disk(5)

Using `cells_in_gel.batch.labels_regions(files)`, segment max projections, label objects, and extract region properties. The output `regions` will give a dictionary containing dataframes of properties for each max intensity projection. 

In [ ]:
regions = batch.labels_regions(files, selem, min_size=350)

Here is the head for the first max intensity projection dataframe in our list of `files`.

In [ ]:
regions[files[0]].head()

## Extracting Average Values from Max Intesnity Projections

Create a list of `titles` that describe categories between each underscore in filenames. Create a list of average `properties` to extract from the max intensity projection regions dataframe.

In [ ]:
titles = ['celltype', 'ecmtype_conc', 'rgd_conc', 'mag', 'img_num']
properties = ['area', 'eccentricity', 'extent']

Using `batch.image_summaries(files, properties, titles, regions)`, extract a summary dataframe that includes average and variance properties from each max intensity projection. See the `summary` head below. 

In [ ]:
summary = batch.image_summaries(files, properties, titles, regions)
summary.head()

Further analysis of grouping, etc. will be done using the `summary` dataframe. At this point, you can use `summary.to_csv('imagesummaries_green.csv', index=False)` for creating a CSV of this dataframe. Further analysis can be done in excel if preferred at this point.  

In [ ]:
summary.to_csv('image_summaries.csv')

## Graphing Groups

Graphing of the `summary` dataframe requires further grouping of the data. The `batch.groups(summary)` function groups according to unique combinations of cell type, ECM type, and RGD concentration. This results in 20 groups that will be placed in the final dataframe shown below. The column titled index indicates the unique group combination of cell type, ecm type, and RGD type. 

In [ ]:
final = batch.groups(summary)
final

Properties and variance of these properties should now be defined for grouping the data into lists appropriate for graphing. 

In [ ]:
properties = ['area', 'eccentricity', 'extent']
variance = ['area_var', 'eccentricity_var', 'extent_var']

Lists of properties and groups can now be made for graphing. Here is what each group represents:
* Group 1: I61QTTA cells with no RGD
* Group 2: I61QTTA cells with RGD
* Group 3: NTG cells with no RGD
* Group 4: NTG cells with RGD

Within each group, contains a list of properties each entry corresponding to `properties` previously defined:
* First entry: area
* Second entry: eccentricity
* Third entry: extent

In [ ]:
g1, g2, g3, g4 = batch.groupby_property(final, properties)

A similar grouping can be done for the error bars of each group. Error bars area calculated as 2 * SEM (Standard Error of the Mean). 

In [ ]:
yerr_g1, yerr_g2, yerr_g3, yerr_g4 = batch.group_sem(final, variance)

Finally, a graph can be constructed using each of the groups we defined and their corresponding SEM. 

In [ ]:
batch.graph(g1, g2, g3, g4, yerr_g1, yerr_g2, yerr_g3, yerr_g4)